<a href="https://colab.research.google.com/github/ElenaNtm/power-thefts-investigation/blob/main/hedno_prob_model(8).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Libraries

In [ ]:
import pandas as pd 
import numpy as np
import math
import operator 
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.metrics import balanced_accuracy_score
import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.compose import make_column_selector as selector
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.linear_model import RidgeClassifier
from imblearn.ensemble import RUSBoostClassifier

In [ ]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

In [ ]:
# explicitly require this experimental feature
from sklearn.experimental import enable_hist_gradient_boosting 
# now you can import normally from ensemble
from sklearn.ensemble import HistGradientBoostingClassifier

/usr/local/lib/python3.9/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


In [ ]:
import joblib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#scoring 
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

#Data

In [ ]:
path = "/content/drive/MyDrive/HEDNO/new_test.csv"
test_data = pd.read_csv(path)
test_data = test_data.drop(['Unnamed: 0'],axis=1)
test_data.head()

,ACCT_NBR,SUCCESSOR,VOLTAGE,PARNO,XRHSH,CONTRACT_CAPACITY,ACCT_CONTROL,TOTAL_REQUEST_TYPE,LAST_REQUEST_TYPE,TOTAL_SUPPLIER,LAST_SUPPLIER,TOTAL_MS_METER_NBR
0,000A31FE38C6F648845941B642E287FE0200000000006D...,1,0.0,0,1,12,0,1.0,3.0,1.0,23.0,1.0
1,000A31FE38C6F648845941B642E287FE0200000000006E...,1,0.0,0,1,12,0,1.0,3.0,1.0,23.0,1.0
2,000A31FE38C6F648845941B642E287FE020000000001BF...,2,0.0,2,1,25,0,1.0,3.0,1.0,23.0,1.0
3,000A31FE38C6F648845941B642E287FE02000000000357...,1,0.0,0,1,12,0,1.0,3.0,1.0,23.0,1.0
4,000A31FE38C6F648845941B642E287FE02000000000536...,1,0.0,0,1,12,0,1.0,3.0,1.0,23.0,1.0


In [ ]:
#test_data.describe

In [ ]:
#test_data.drop(['CSS_MS_HS_USE','MEASUREMENT_DATE'], axis=1, inplace = True)

In [ ]:
#test_data = test_data.drop_duplicates(subset=['ACCT_NBR', 'SUCCESSOR'], keep='last')

In [ ]:
#test_data.head()

In [ ]:
#test_data.describe

In [ ]:
path = "/content/drive/MyDrive/HEDNO/result_train.csv"
train_data = pd.read_csv(path)
train_data =train_data.drop(['MEASUREMENT_DATE'],axis=1)
train_data =train_data.drop(['CSS_MS_HS_USE'],axis=1)
train_data =train_data.drop(['START_DATE'],axis=1)
train_data =train_data.drop(['SUPPLIER'],axis=1)
train_data = train_data.drop_duplicates()
train_data.head()

,ACCT_NBR,SUCCESSOR,VOLTAGE,PARNO,XRHSH,CONTRACT_CAPACITY,ACCT_CONTROL,TOTAL_REQUEST_TYPE,LAST_REQUEST_TYPE,TOTAL_SUPPLIER,LAST_SUPPLIER,TOTAL_MS_METER_NBR,target
0,000A31FE38C6F648845941B642E287FE0200000000008D...,5,0,2,2,25,0,4,2,1,15,1,0
12,000A31FE38C6F648845941B642E287FE0200000000008E...,1,0,0,1,12,0,1,3,1,15,1,0
20,000A31FE38C6F648845941B642E287FE0200000000016E...,13,0,0,1,12,0,1,3,1,15,1,0
33,000A31FE38C6F648845941B642E287FE020000000001DB...,3,0,0,1,12,0,9,2,1,15,1,0
46,000A31FE38C6F648845941B642E287FE02000000000297...,2,0,2,1,25,0,1,4,1,15,1,0


In [ ]:
#train_data.columns

In [ ]:
#train_data.drop([])

In [ ]:
#train_data.duplicated(keep='last')

#Index

In [ ]:
id_acct = test_data['ACCT_NBR']
id_succ = test_data['SUCCESSOR']
sub = pd.DataFrame()
sub['ACCT_NBR'] = id_acct
sub['SUCCESSOR'] = id_succ
sub.head()

,ACCT_NBR,SUCCESSOR
0,000A31FE38C6F648845941B642E287FE0200000000006D...,1
1,000A31FE38C6F648845941B642E287FE0200000000006E...,1
2,000A31FE38C6F648845941B642E287FE020000000001BF...,2
3,000A31FE38C6F648845941B642E287FE02000000000357...,1
4,000A31FE38C6F648845941B642E287FE02000000000536...,1


In [ ]:
train_data = train_data.set_index(['ACCT_NBR', 'SUCCESSOR'])
test_data = test_data.set_index(['ACCT_NBR', 'SUCCESSOR'])

In [ ]:
sub = sub.set_index(['ACCT_NBR', 'SUCCESSOR'])

In [ ]:
sub.head()

,
ACCT_NBR,SUCCESSOR
000A31FE38C6F648845941B642E287FE0200000000006DA0F81C7B34516BCA84E6BCA0AA9B50D8259400B2DCE41040ABA5315450F7A570DBC66F778B3CD920C475F6EA9D,1
000A31FE38C6F648845941B642E287FE0200000000006E6FF09E2B4B75110DFE70B0121E6696E3AFF9E02C61AB90436475F6486E3AB9E89C4CEEBBA660A506B83C95475E,1
000A31FE38C6F648845941B642E287FE020000000001BF1BACB232B9758BE9787F85BC3C6C186314FDD2257714190DD43B2B6F721FC865BAD5CD985A12A69FABFBF642C3,2
000A31FE38C6F648845941B642E287FE020000000003575CE1056D200939ECBB4EEA729A68F66DE14FEBD4795D07D967FC3A96150C592499A0B465386923979CFB5BA8FB,1
000A31FE38C6F648845941B642E287FE0200000000053609F480423B3806FD211D94BE7EFE56BB61A4C24ADE61B30B67756D1E9D1F6229386F4A68F09844F1926CDDAD48,1


In [ ]:
train_data.head()

,,VOLTAGE,PARNO,XRHSH,CONTRACT_CAPACITY,ACCT_CONTROL,TOTAL_REQUEST_TYPE,LAST_REQUEST_TYPE,TOTAL_SUPPLIER,LAST_SUPPLIER,TOTAL_MS_METER_NBR,target
ACCT_NBR,SUCCESSOR,,,,,,,,,,,
000A31FE38C6F648845941B642E287FE0200000000008D3F160EBFD6AB4F42A9C12D32B9040BC182F649B304179696516630F3AC54BE7C2F7A85C8484FFF4A09FAA497AF,5,0,2,2,25,0,4,2,1,15,1,0
000A31FE38C6F648845941B642E287FE0200000000008EB578F7B9D9030CF23D3FE10CEF5B0147F915BD845358B8349EB8C605038714C1662810849DF046563E30DD4E12,1,0,0,1,12,0,1,3,1,15,1,0
000A31FE38C6F648845941B642E287FE0200000000016EFD4EAC63D9EAE40DBD382D7784199CE43FEE2FE782860583283A0C5C93ADD7C44351EB41B8C090E3E68C2703B4,13,0,0,1,12,0,1,3,1,15,1,0
000A31FE38C6F648845941B642E287FE020000000001DB083F761E1521BF481F87A9451248D0F4B563F1B54429BF18D7E79A763029E596FD54ABCDC7C75C7A4B61964856,3,0,0,1,12,0,9,2,1,15,1,0
000A31FE38C6F648845941B642E287FE020000000002974B9AB8CB064E84D4F506B620099EDB2B8496C496DDA7045A4147995E2109719A1A3BCEA48F311745BD63FCCBFB,2,0,2,1,25,0,1,4,1,15,1,0


# Balanced Random Forest fit and predict

In [ ]:
X_train = train_data.drop('target',axis=1)
X_train.head()

,,VOLTAGE,PARNO,XRHSH,CONTRACT_CAPACITY,ACCT_CONTROL,TOTAL_REQUEST_TYPE,LAST_REQUEST_TYPE,TOTAL_SUPPLIER,LAST_SUPPLIER,TOTAL_MS_METER_NBR
ACCT_NBR,SUCCESSOR,,,,,,,,,,
000A31FE38C6F648845941B642E287FE0200000000008D3F160EBFD6AB4F42A9C12D32B9040BC182F649B304179696516630F3AC54BE7C2F7A85C8484FFF4A09FAA497AF,5,0,2,2,25,0,4,2,1,15,1
000A31FE38C6F648845941B642E287FE0200000000008EB578F7B9D9030CF23D3FE10CEF5B0147F915BD845358B8349EB8C605038714C1662810849DF046563E30DD4E12,1,0,0,1,12,0,1,3,1,15,1
000A31FE38C6F648845941B642E287FE0200000000016EFD4EAC63D9EAE40DBD382D7784199CE43FEE2FE782860583283A0C5C93ADD7C44351EB41B8C090E3E68C2703B4,13,0,0,1,12,0,1,3,1,15,1
000A31FE38C6F648845941B642E287FE020000000001DB083F761E1521BF481F87A9451248D0F4B563F1B54429BF18D7E79A763029E596FD54ABCDC7C75C7A4B61964856,3,0,0,1,12,0,9,2,1,15,1
000A31FE38C6F648845941B642E287FE020000000002974B9AB8CB064E84D4F506B620099EDB2B8496C496DDA7045A4147995E2109719A1A3BCEA48F311745BD63FCCBFB,2,0,2,1,25,0,1,4,1,15,1


In [ ]:
y_train = train_data['target']
y_train.head()

ACCT_NBR                                                                                                                                  SUCCESSOR
000A31FE38C6F648845941B642E287FE0200000000008D3F160EBFD6AB4F42A9C12D32B9040BC182F649B304179696516630F3AC54BE7C2F7A85C8484FFF4A09FAA497AF  5            0
000A31FE38C6F648845941B642E287FE0200000000008EB578F7B9D9030CF23D3FE10CEF5B0147F915BD845358B8349EB8C605038714C1662810849DF046563E30DD4E12  1            0
000A31FE38C6F648845941B642E287FE0200000000016EFD4EAC63D9EAE40DBD382D7784199CE43FEE2FE782860583283A0C5C93ADD7C44351EB41B8C090E3E68C2703B4  13           0
000A31FE38C6F648845941B642E287FE020000000001DB083F761E1521BF481F87A9451248D0F4B563F1B54429BF18D7E79A763029E596FD54ABCDC7C75C7A4B61964856  3            0
000A31FE38C6F648845941B642E287FE020000000002974B9AB8CB064E84D4F506B620099EDB2B8496C496DDA7045A4147995E2109719A1A3BCEA48F311745BD63FCCBFB  2            0
Name: target, dtype: int64

In [ ]:
brf = BalancedRandomForestClassifier(n_estimators=200,random_state=42, n_jobs=-1)
brf.fit(X_train, y_train)

/usr/local/lib/python3.9/dist-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function delayed is deprecated; The function `delayed` has been moved from `sklearn.utils.fixes` to `sklearn.utils.parallel`. This import path will be removed in 1.5.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/utils/parallel.py:114: UserWarning: `sklearn.utils.parallel.delayed` should be used with `sklearn.utils.parallel.Parallel` to make it possible to propagate the scikit-learn configuration of the current thread to the joblib workers.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/utils/parallel.py:114: UserWarning: `sklearn.utils.parallel.delayed` should be used with `sklearn.utils.parallel.Parallel` to make it possible to propagate the scikit-learn configuration of the current thread to the joblib workers.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/utils/parallel.py:114: UserWarning: `sklearn.utils.parallel.de

BalancedRandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=42)

In [ ]:
predictions = brf.predict_proba(test_data)

# saving the predictions

In [ ]:
sub[['negative','positive']] = predictions
sub.head()

,,negative,positive
ACCT_NBR,SUCCESSOR,,
000A31FE38C6F648845941B642E287FE0200000000006DA0F81C7B34516BCA84E6BCA0AA9B50D8259400B2DCE41040ABA5315450F7A570DBC66F778B3CD920C475F6EA9D,1,0.842529,0.157471
000A31FE38C6F648845941B642E287FE0200000000006E6FF09E2B4B75110DFE70B0121E6696E3AFF9E02C61AB90436475F6486E3AB9E89C4CEEBBA660A506B83C95475E,1,0.842529,0.157471
000A31FE38C6F648845941B642E287FE020000000001BF1BACB232B9758BE9787F85BC3C6C186314FDD2257714190DD43B2B6F721FC865BAD5CD985A12A69FABFBF642C3,2,1.000000,0.000000
000A31FE38C6F648845941B642E287FE020000000003575CE1056D200939ECBB4EEA729A68F66DE14FEBD4795D07D967FC3A96150C592499A0B465386923979CFB5BA8FB,1,0.842529,0.157471
000A31FE38C6F648845941B642E287FE0200000000053609F480423B3806FD211D94BE7EFE56BB61A4C24ADE61B30B67756D1E9D1F6229386F4A68F09844F1926CDDAD48,1,0.842529,0.157471


In [ ]:
print(f'\033[94mNumber missing values in predictions: {sum(sub.isna().sum())}')

Number missing values in predictions: 0


In [ ]:
sub.describe()

,negative,positive
count,137534.000000,137534.000000
mean,0.771470,0.228530
std,0.168966,0.168966
min,0.007218,0.000000
25%,0.772661,0.157471
50%,0.842529,0.157471
75%,0.842529,0.227339
max,1.000000,0.992782


In [ ]:
path = "/content/drive/MyDrive/HEDNO/test/successors_TEST.csv"
successors_TEST = pd.read_csv(path, sep= '|')
successors_TEST.head()

,ACCT_NBR,SUCCESSOR,PREDICTION
0,000A31FE38C6F648845941B642E287FE020000008FD392...,3,NaN
1,000A31FE38C6F648845941B642E287FE0200000044A036...,5,NaN
2,000A31FE38C6F648845941B642E287FE0200000029FF97...,5,NaN
3,000A31FE38C6F648845941B642E287FE02000000BF3583...,3,NaN
4,000A31FE38C6F648845941B642E287FE02000000A0B356...,3,NaN


In [ ]:
successors_TEST.describe()

,SUCCESSOR,PREDICTION
count,69400.000000,0.0
mean,2.757061,NaN
std,2.061498,NaN
min,1.000000,NaN
25%,1.000000,NaN
50%,2.000000,NaN
75%,3.000000,NaN
max,23.000000,NaN


In [ ]:
successors_TEST = successors_TEST.set_index(['ACCT_NBR', 'SUCCESSOR'])

In [ ]:
successors_TEST = pd.merge(successors_TEST,sub['positive'], how='left',left_index=True,right_index=True) #, left_on=['ACCT_NBR','SUCCESSOR'], right_on=['ACCT_NBR','SUCCESSOR']

In [ ]:
successors_TEST.head()

,,PREDICTION,positive
ACCT_NBR,SUCCESSOR,,
000A31FE38C6F648845941B642E287FE020000008FD392D322FE12E10F91153CFE112E6C493AA390347C02BDD6519D6BB97D9E020E4FCC58FB7BD13C9858C386A2B504B8,3,NaN,0.386099
000A31FE38C6F648845941B642E287FE0200000044A036CD3A88446FFB86F803C27B8EEDF7E61E333180CE443E73D172FFF440D025EF91CE415C40F206DCCA0A4169FDF6,5,NaN,0.191848
000A31FE38C6F648845941B642E287FE0200000029FF97D8CC67EE0B5D0D18E0C058017AEC80FE21DBA335B6418851B59149D16EDCFF6C0F3A70A9DAA087B4C535A7CCA3,5,NaN,0.759612
000A31FE38C6F648845941B642E287FE02000000BF35833136D1D51BACB49B75AA1F3A0C878C0B62082DB73E25DB011DA4909D22DA26A3DABB9474F92F267F0170CFE78B,3,NaN,0.157471
000A31FE38C6F648845941B642E287FE02000000A0B3566A3149131062B7E049B03313FFA4A097EE07248BBD71AD7CC061E9A2D59C5BDD332819E4E8AAE9F27E975B0D9E,3,NaN,0.193724


In [ ]:
successors_TEST.describe()

,PREDICTION,positive
count,0.0,69400.000000
mean,NaN,0.261201
std,NaN,0.199729
min,NaN,0.000000
25%,NaN,0.157471
50%,NaN,0.157471
75%,NaN,0.375979
max,NaN,0.992782


In [ ]:
successors_TEST.isnull().count()

PREDICTION    69400
positive      69400
dtype: int64

In [ ]:
successors_TEST.drop(['PREDICTION'],axis = 1, inplace = True)

In [ ]:
successors_TEST.rename({"positive": "PREDICTION"}, axis=1, inplace=True)
print(successors_TEST.dtypes)

PREDICTION    float64
dtype: object


In [ ]:
successors_TEST.head()

,,PREDICTION
ACCT_NBR,SUCCESSOR,
000A31FE38C6F648845941B642E287FE020000008FD392D322FE12E10F91153CFE112E6C493AA390347C02BDD6519D6BB97D9E020E4FCC58FB7BD13C9858C386A2B504B8,3,0.386099
000A31FE38C6F648845941B642E287FE0200000044A036CD3A88446FFB86F803C27B8EEDF7E61E333180CE443E73D172FFF440D025EF91CE415C40F206DCCA0A4169FDF6,5,0.191848
000A31FE38C6F648845941B642E287FE0200000029FF97D8CC67EE0B5D0D18E0C058017AEC80FE21DBA335B6418851B59149D16EDCFF6C0F3A70A9DAA087B4C535A7CCA3,5,0.759612
000A31FE38C6F648845941B642E287FE02000000BF35833136D1D51BACB49B75AA1F3A0C878C0B62082DB73E25DB011DA4909D22DA26A3DABB9474F92F267F0170CFE78B,3,0.157471
000A31FE38C6F648845941B642E287FE02000000A0B3566A3149131062B7E049B03313FFA4A097EE07248BBD71AD7CC061E9A2D59C5BDD332819E4E8AAE9F27E975B0D9E,3,0.193724


In [ ]:
#index = successors_TEST.index
#number_of_rows = len(index)

In [ ]:
#number_of_rows

In [ ]:
total = successors_TEST.isnull().sum().sort_values(ascending=False)
percent = (successors_TEST.isnull().sum()/successors_TEST.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
PREDICTION,0,0.0


In [ ]:
path = "/content/drive/MyDrive/HEDNO/Submission2022.csv"
with open(path, 'w', encoding = 'utf-8-sig') as f:
  successors_TEST.to_csv(f)

In [ ]:
#successors_TEST[successors_TEST['PREDICTION'].isnull()].index.tolist()